In [68]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import Precision as KerasPrecision, Recall as KerasRecall # Alias untuk menghindari konflik nama
from tensorflow.keras.regularizers import l2 # Untuk L2 Regularization
from sklearn.metrics.pairwise import euclidean_distances

# Impor untuk metrik evaluasi peringkat
from sklearn.metrics import ndcg_score
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [69]:
"""# 0. Konfigurasi dan Variabel Global"""
# Variabel global untuk model dan encoder yang telah dilatih
trained_ncf_model = None
le_artists_global = None
le_song_id_global = None # Encoder untuk ID lagu asli ke indeks numerik
df_global = None # Menyimpan dataframe yang sudah diproses secara global

# K untuk evaluasi peringkat
K_EVALUATION = 10

# Load Dataset

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
# Load data
df = pd.read_csv("/content/drive/MyDrive/Music Recommendation System using Spotify New Dataset.csv")


In [71]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170653 entries, 0 to 170652
Data columns (total 31 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   valence                                 170653 non-null  float64
 1   year                                    170653 non-null  int64  
 2   acousticness                            170653 non-null  float64
 3   artists                                 170653 non-null  object 
 4   danceability                            170653 non-null  float64
 5   duration_ms                             170653 non-null  int64  
 6   energy                                  170653 non-null  float64
 7   explicit                                170653 non-null  int64  
 8   id                                      170653 non-null  object 
 9   instrumentalness                        170653 non-null  float64
 10  key                                     1706

,valence,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,duration_min
count,170653.000000,170653.000000,170653.000000,170653.000000,1.706530e+05,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000,170653.000000
mean,0.528587,1976.787241,0.502115,0.537396,2.309483e+05,0.482389,0.084575,0.167010,5.199844,0.205839,-11.467990,0.706902,31.431794,0.098393,116.861590,3.849139
std,0.263171,25.917853,0.376032,0.176138,1.261184e+05,0.267646,0.278249,0.313475,3.515094,0.174805,5.697943,0.455184,21.826615,0.162740,30.708533,2.101974
min,0.000000,1921.000000,0.000000,0.000000,5.108000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.085133
25%,0.317000,1956.000000,0.102000,0.415000,1.698270e+05,0.255000,0.000000,0.000000,2.000000,0.098800,-14.615000,0.000000,11.000000,0.034900,93.421000,2.830450
50%,0.540000,1977.000000,0.516000,0.548000,2.074670e+05,0.471000,0.000000,0.000216,5.000000,0.136000,-10.580000,1.000000,33.000000,0.045000,114.729000,3.457783
75%,0.747000,1999.000000,0.893000,0.668000,2.624000e+05,0.703000,0.000000,0.102000,8.000000,0.261000,-7.183000,1.000000,48.000000,0.075600,135.537000,4.373333
max,1.000000,2020.000000,0.996000,0.988000,5.403500e+06,1.000000,1.000000,1.000000,11.000000,1.000000,3.855000,1.000000,100.000000,0.970000,243.507000,90.058333


Melakukan load dataset yang berada di google drive

# Cleaning Data

In [72]:
df = df.dropna()
df = df.reset_index(drop=True)
print("\nInformasi Dataset Setelah Pembersihan:")
df.info()


Informasi Dataset Setelah Pembersihan:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165235 entries, 0 to 165234
Data columns (total 31 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   valence                                 165235 non-null  float64
 1   year                                    165235 non-null  int64  
 2   acousticness                            165235 non-null  float64
 3   artists                                 165235 non-null  object 
 4   danceability                            165235 non-null  float64
 5   duration_ms                             165235 non-null  int64  
 6   energy                                  165235 non-null  float64
 7   explicit                                165235 non-null  int64  
 8   id                                      165235 non-null  object 
 9   instrumentalness                        165235 non-null  float64
 10  key 

Dataset sudah bersih sehingga tidak perlu dilakukan cleaning secara ekstensif selain menghapus beberapa kolom yang memiliki nilai null agar model dapat bekerja dengan lebih optimal.

# Pre-processing Data

In [73]:
def engineer_features(df_to_engineer):
    # 1 - Konversi durasi_ms ke durasi_min
    df_to_engineer['duration_min'] = df_to_engineer['duration_ms'] / 60000

    # 2 - Kategori berdasarkan valence
    df_to_engineer['type_of_song_by_valence'] = pd.cut(
        df_to_engineer['valence'],
        bins=[-float('inf'), 0.3, 0.6, float('inf')],
        labels=['Sad song', 'Balance song between sadness and joy', 'Happy song'],
        right=False
    )

    # 3 - Kategori berdasarkan acousticness
    df_to_engineer['type_of_song_by_acousticness'] = pd.cut(
        df_to_engineer['acousticness'],
        bins=[-float('inf'), 0.3, 0.6, float('inf')],
        labels=['Highly electronic song', 'Mix of acoustic and electronic instruments', 'Mostly acoustic song'],
        right=False
    )

    # 4 - Kategori berdasarkan danceability
    df_to_engineer['type_of_song_by_danceability'] = pd.cut(
        df_to_engineer['danceability'],
        bins=[-float('inf'), 0.4, 0.7, float('inf')],
        labels=['Not suitable for dance', 'Medium-paced song', 'Highly danceable song'],
        right=False
    )

    # 5 - Kategori berdasarkan energy
    df_to_engineer['Energy_Level_of_the_Song'] = pd.cut(
        df_to_engineer['energy'],
        bins=[-float('inf'), 0.3, 0.6, float('inf')],
        labels=['Slow song', 'Medium-energy song', 'High-energy song (Enthusiastic)'],
        right=False
    )

    # 6 - Kategori berdasarkan instrumentalness
    df_to_engineer['type_of_song_by_instrumentalness'] = pd.cut(
        df_to_engineer['instrumentalness'],
        bins=[-float('inf'), 0.3, 0.7, float('inf')],
        labels=['Song clearly contains lyrics', 'Song contain some words', 'Nearly purely instrumental song'],
        right=False
    )

    # 7 - Kategori berdasarkan liveness
    df_to_engineer['type_of_song_by_liveness'] = pd.cut(
        df_to_engineer['liveness'],
        bins=[-float('inf'), 0.3, 0.7, float('inf')],
        labels=['Pure studio recordings', 'Records may have a simple reaction with the audience', 'Clearly a live recording with an audience'],
        right=False
    )

    # 8 - Kategori berdasarkan loudness
    df_to_engineer['type_of_song_by_loudness(dB)'] = pd.cut(
        df_to_engineer['loudness'],
        bins=[-60, -30, -5, float('inf')],
        labels=['Quieter Sound', 'Quiet song', 'Loud song'],
        right=False
    )
     # Mengisi NaN yang mungkin muncul jika ada nilai di luar bins (misal loudness > -5)
    df_to_engineer['type_of_song_by_loudness(dB)'] = df_to_engineer['type_of_song_by_loudness(dB)'].cat.add_categories('Very Quiet or Unspecified').fillna('Very Quiet or Unspecified')


    # 9 - Kategori berdasarkan mode
    df_to_engineer['type_of_song_by_mode'] = df_to_engineer['mode'].apply(lambda x: 'Happy tone (Major)' if x == 1 else 'Sad tone (Minor)')

    # 10 - Kategori berdasarkan speechiness
    df_to_engineer['type_of_song_by_speechiness'] = pd.cut(
        df_to_engineer['speechiness'],
        bins=[-float('inf'), 0.3, 0.7, float('inf')],
        labels=['Mostly instrumental song', 'Song with some speech', 'Podcasts or speech-heavy rap'],
        right=False
    )

    # 11 - Kategori berdasarkan tempo
    df_to_engineer['type_of_tempo_in_Beats_Per_Minute(BPM)'] = pd.cut(
        df_to_engineer['tempo'],
        bins=[40, 80, 120, 180, float('inf')],
        labels=['Slow song', 'Medium-paced song', 'Fast-paced song', 'Very Fast-paced song'],
        right=False
    )
    # Mengisi NaN yang mungkin muncul jika ada nilai di luar bins
    df_to_engineer['type_of_tempo_in_Beats_Per_Minute(BPM)'] = df_to_engineer['type_of_tempo_in_Beats_Per_Minute(BPM)'].cat.add_categories('Very Slow or Unspecified').fillna('Very Slow or Unspecified')

    return df_to_engineer

df = engineer_features(df.copy())
print("\nDataFrame setelah Rekayasa Fitur (contoh beberapa kolom baru):")
print(df[['name', 'valence', 'type_of_song_by_valence', 'acousticness', 'type_of_song_by_acousticness']].head())


"""# 3. Pra-pemrosesan Data"""

# FITUR NUMERIK YANG AKAN DINORMALISASI
NUMERIC_FEATURES_FOR_SCALING = [
    'valence', 'danceability', 'energy', 'acousticness',
    'instrumentalness', 'liveness', 'loudness', 'tempo', 'duration_min', 'popularity'
]

# FITUR YANG DIGUNAKAN UNTUK PREDIKSI NCF (FITUR TAMBAHAN)
NCF_ADDITIONAL_FEATURES = ['valence', 'danceability', 'energy', 'acousticness', 'instrumentalness']

# FITUR YANG DIGUNAKAN UNTUK CONTENT-BASED FILTERING
CBF_FEATURES = ['valence', 'danceability', 'energy', 'acousticness', 'instrumentalness', 'liveness']


def prepare_data(df_to_prepare):
    global le_artists_global, le_song_id_global # Menggunakan variabel global

    # Praproses fitur numerik
    scaler = MinMaxScaler()
    df_to_prepare[NUMERIC_FEATURES_FOR_SCALING] = scaler.fit_transform(df_to_prepare[NUMERIC_FEATURES_FOR_SCALING])

    # Encoding 'artists'
    le_artists_global = LabelEncoder()
    df_to_prepare['artist_encoded'] = le_artists_global.fit_transform(df_to_prepare['artists'])

    # Encoding 'id' lagu asli menjadi indeks numerik 0 sampai N-1
    le_song_id_global = LabelEncoder()
    df_to_prepare['song_index_encoded'] = le_song_id_global.fit_transform(df_to_prepare['id'])


    # Rekayasa Fitur: Popularity binning untuk target NCF
    median_popularity = df_to_prepare['popularity'].median() # Popularity di sini sudah dinormalisasi (0-1)
    df_to_prepare['is_popular_binary'] = (df_to_prepare['popularity'] > median_popularity).astype(int)


    # Membuat kolom 'popularity_bin' (kategorikal) jika diperlukan untuk analisis lain, bukan target NCF
    return df_to_prepare

df = prepare_data(df.copy())
df_global = df.copy() # Simpan df yang sudah diproses secara global

print("\nDataFrame setelah pra-pemrosesan (contoh beberapa kolom):")
print(df[['name', 'artists', 'artist_encoded', 'id', 'song_index_encoded', 'popularity', 'is_popular_binary'] + NCF_ADDITIONAL_FEATURES].head())


def split_data(df_to_split):
    train_df, test_df = train_test_split(df_to_split, test_size=0.2, random_state=42, stratify=df_to_split['is_popular_binary'])
    return train_df, test_df

train_df, test_df = split_data(df)
print(f"\nUkuran data latih: {train_df.shape}")
print(f"Ukuran data uji: {test_df.shape}")




DataFrame setelah Rekayasa Fitur (contoh beberapa kolom baru):
                                                name  valence  \
0  Piano Concerto No. 3 in D Minor, Op. 30: III. ...   0.0594   
1                            Clancy Lowered the Boom   0.9630   
2                                          Gati Bali   0.0394   
3                                          Danny Boy   0.1650   
4                        When Irish Eyes Are Smiling   0.2530   

  type_of_song_by_valence  acousticness type_of_song_by_acousticness  
0                Sad song         0.982         Mostly acoustic song  
1              Happy song         0.732         Mostly acoustic song  
2                Sad song         0.961         Mostly acoustic song  
3                Sad song         0.967         Mostly acoustic song  
4                Sad song         0.957         Mostly acoustic song  

DataFrame setelah pra-pemrosesan (contoh beberapa kolom):
                                                name  \
0  P

### Data Preparation

#### 1. Rekayasa Fitur (`engineer_features()`)

Fungsi ini bertujuan untuk menciptakan fitur-fitur baru yang lebih informatif dari fitur yang sudah ada, yang berpotensi meningkatkan performa model.

* **Konversi Durasi:**
    * Mengubah kolom `duration_ms` (durasi dalam milidetik) menjadi `duration_min` (durasi dalam menit) untuk skala yang mungkin lebih intuitif.
* **Pembentukan Kategori Fitur Audio:**
    * Membuat 11 kolom kategorikal baru berdasarkan nilai-nilai fitur audio numerik. Proses ini menggunakan fungsi `pd.cut` untuk membagi rentang nilai fitur menjadi beberapa bin (kategori) yang telah ditentukan. Fitur-fitur yang dikategorikan meliputi:
        * `valence` $\rightarrow$ `type_of_song_by_valence` (misalnya, "Sad song", "Happy song")
        * `acousticness` $\rightarrow$ `type_of_song_by_acousticness` (misalnya, "Highly electronic song", "Mostly acoustic song")
        * `danceability` $\rightarrow$ `type_of_song_by_danceability` (misalnya, "Not suitable for dance", "Highly danceable song")
        * `energy` $\rightarrow$ `Energy_Level_of_the_Song` (misalnya, "Slow song", "High-energy song")
        * `instrumentalness` $\rightarrow$ `type_of_song_by_instrumentalness` (misalnya, "Song clearly contains lyrics", "Nearly purely instrumental song")
        * `liveness` $\rightarrow$ `type_of_song_by_liveness` (misalnya, "Pure studio recordings", "Clearly a live recording")
        * `loudness` $\rightarrow$ `type_of_song_by_loudness(dB)` (misalnya, "Quieter Sound", "Loud song"). Dilakukan penanganan nilai NaN jika ada data di luar rentang bin yang ditentukan.
        * `speechiness` $\rightarrow$ `type_of_song_by_speechiness` (misalnya, "Mostly instrumental song", "Podcasts or speech-heavy rap")
        * `tempo` $\rightarrow$ `type_of_tempo_in_Beats_Per_Minute(BPM)` (misalnya, "Slow song", "Fast-paced song"). Dilakukan penanganan nilai NaN.
* **Kategori Berdasarkan Mode:**
    * Mengubah fitur `mode` (0 untuk minor, 1 untuk mayor) menjadi kolom kategorikal `type_of_song_by_mode` ("Sad tone (Minor)", "Happy tone (Major)").

#### 2. Pra-pemrosesan Inti (`prepare_data()`)

Fungsi ini melakukan transformasi data lebih lanjut setelah rekayasa fitur, mempersiapkannya secara spesifik untuk input model.

* **Normalisasi Fitur Numerik:**
    * Menggunakan `MinMaxScaler()` dari `sklearn.preprocessing` untuk menskalakan nilai-nilai fitur numerik ke rentang antara 0 dan 1.
    * Fitur-fitur yang dinormalisasi (sesuai variabel `NUMERIC_FEATURES_FOR_SCALING`) meliputi:
        * `valence`, `danceability`, `energy`, `acousticness`, `instrumentalness`, `liveness`, `loudness`, `tempo`, `duration_min` (hasil rekayasa fitur), dan `popularity`.
* **Encoding Fitur Kategorikal:**
    * **Encoding Artis:** Menggunakan `LabelEncoder()` untuk mengubah nama-nama artis (fitur `artists`) menjadi representasi numerik unik. Hasilnya disimpan dalam kolom baru `artist_encoded` dan objek encoder disimpan secara global di `le_artists_global`.
    * **Encoding ID Lagu:** Menggunakan `LabelEncoder()` untuk mengubah ID lagu asli (fitur `id`) menjadi indeks numerik unik (mulai dari 0 hingga N-1). Ini penting untuk digunakan sebagai input pada layer Embedding item di model NCF. Hasilnya disimpan di kolom `song_index_encoded` dan objek encoder disimpan di `le_song_id_global`.
* **Pembuatan Fitur Target untuk NCF:**
    * Membuat kolom target biner bernama `is_popular_binary`. Sebuah lagu dianggap populer (`1`) jika nilai `popularity`-nya (yang sudah dinormalisasi) lebih besar dari median `popularity` keseluruhan dataset. Jika tidak, maka nilainya `0`. Kolom ini akan digunakan sebagai target prediksi untuk model NCF.

#### 3. Pembagian Data (`split_data()`)

Fungsi ini bertanggung jawab untuk membagi keseluruhan dataset yang telah diproses menjadi dua subset: data latih (training set) dan data uji (test set).

* **Metode Pembagian:** Menggunakan fungsi `train_test_split()` dari `sklearn.model_selection`.
* **Proporsi Pembagian:** Data dibagi dengan proporsi 80% untuk data latih dan 20% untuk data uji (`test_size=0.2`).
* **Reproduktibilitas:** Menggunakan `random_state=42` untuk memastikan bahwa pembagian data selalu sama setiap kali kode dijalankan, sehingga hasil eksperimen dapat direproduksi.
* **Stratifikasi:** Pembagian dilakukan secara stratifikasi berdasarkan kolom `is_popular_binary` (`stratify=df_to_split['is_popular_binary']`). Ini memastikan bahwa proporsi lagu populer dan tidak populer kurang lebih sama di kedua set data (latih dan uji), yang penting untuk melatih dan mengevaluasi model klasifikasi secara seimbang.

# Modelling

In [76]:
## 4.1. Model NCF (Neural Collaborative Filtering)
def create_ncf_model(num_unique_artists, num_unique_songs, latent_dim=64, num_additional_features=5): # latent_dim ditingkatkan
    # Input layers
    user_input = Input(shape=(1,), name='user_input_artist') # Input untuk artist_encoded
    item_input = Input(shape=(1,), name='item_input_song_index') # Input untuk song_index_encoded

    # Embedding layers
    user_embedding = Embedding(num_unique_artists, latent_dim, name='user_embedding', embeddings_regularizer=l2(0.001))(user_input)
    item_embedding = Embedding(num_unique_songs, latent_dim, name='item_embedding', embeddings_regularizer=l2(0.001))(item_input)

    # Flatten embeddings
    user_flat = Flatten(name='user_flatten')(user_embedding)
    item_flat = Flatten(name='item_flatten')(item_embedding)

    # Input untuk fitur tambahan lagu
    additional_features_input = Input(shape=(num_additional_features,), name='additional_features_input')

    # Concatenate embeddings dengan fitur tambahan
    concat = Concatenate(name='concatenate_layer')([user_flat, item_flat, additional_features_input])

    # Dense layers dengan L2 Regularization dan Dropout yang disesuaikan
    fc1 = Dense(128, activation='relu', name='fc1', kernel_regularizer=l2(0.001))(concat)
    dropout1 = Dropout(0.4, name='dropout1')(fc1)
    fc2 = Dense(64, activation='relu', name='fc2', kernel_regularizer=l2(0.001))(dropout1)
    dropout2 = Dropout(0.2, name='dropout2')(fc2)
    output = Dense(1, activation='sigmoid', name='output_layer')(dropout2)

    model = Model(
        inputs=[user_input, item_input, additional_features_input],
        outputs=output
    )

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            KerasPrecision(name='precision'),
            KerasRecall(name='recall')
        ]
    )
    return model

# Mendapatkan jumlah unik artist dan lagu dari keseluruhan dataset yang sudah diproses
num_artists = df_global['artist_encoded'].nunique()
num_songs = df_global['song_index_encoded'].nunique()

# Inisialisasi model NCF (belum dilatih)
ncf_model_instance = create_ncf_model(num_artists, num_songs, num_additional_features=len(NCF_ADDITIONAL_FEATURES))
print("\nRingkasan Arsitektur Model NCF:")
ncf_model_instance.summary()


## 4.2. Model Content-Based Filtering (CBF)

def content_based_similarity_scores(df_cb, target_song_id_original, features_for_cbf):
    """Menghitung skor kemiripan (jarak terbalik) untuk CBF."""
    if target_song_id_original not in df_cb['id'].values:
        print(f"ID Lagu '{target_song_id_original}' tidak ditemukan untuk CBF.")
        return pd.Series(dtype='float64')

    target_song_features_df = df_cb[df_cb['id'] == target_song_id_original][features_for_cbf]
    if target_song_features_df.empty:
        return pd.Series(dtype='float64')

    target_song_features_vector = target_song_features_df.values # Bentuknya (1, n_features)

    all_songs_features_matrix = df_cb[features_for_cbf].values # Bentuknya (n_total_songs, n_features)

    # Hitung jarak Euclidean antara lagu target dan semua lagu lain
    distances_array = euclidean_distances(target_song_features_vector, all_songs_features_matrix).flatten()

    # Buat Series dengan indeks yang benar
    distances = pd.Series(distances_array, index=df_cb.index)

    # Konversi jarak menjadi skor kemiripan (1 / (1 + jarak)) -> semakin besar semakin mirip
    similarity_scores = 1 / (1 + distances)
    return similarity_scores


def recommend_content_based_pure(df_rec_cb, target_song_id_original, top_k=10):
    """Memberikan Top-N rekomendasi murni berdasarkan Content-Based Filtering."""
    if target_song_id_original not in df_rec_cb['id'].values:
        print(f"Lagu dengan ID '{target_song_id_original}' tidak ditemukan untuk rekomendasi CBF.")
        return pd.DataFrame()

    # Hitung skor kemiripan
    cb_scores = content_based_similarity_scores(df_rec_cb, target_song_id_original, CBF_FEATURES)
    if cb_scores.empty:
        return pd.DataFrame()

    # Urutkan berdasarkan skor kemiripan (descending)
    top_indices = cb_scores.sort_values(ascending=False).index

    # Dapatkan lagu teratas, kecualikan lagu input jika ada
    input_song_idx = df_rec_cb[df_rec_cb['id'] == target_song_id_original].index
    if not input_song_idx.empty:
        top_indices = top_indices.drop(input_song_idx, errors='ignore')

    recommendations_df = df_rec_cb.iloc[top_indices[:top_k]][['id', 'name', 'artists', 'popularity'] + CBF_FEATURES]
    return recommendations_df


## 4.3. Fungsi Rekomendasi Tambahan untuk NCF (Top-N)

def recommend_ncf_pure(df_ncf_rec, artist_id_encoded, top_k=10, trained_model=None):
    """Memberikan Top-N rekomendasi murni dari NCF untuk seorang artis."""
    global trained_ncf_model # Menggunakan model NCF global yang sudah dilatih
    if trained_model is None:
        trained_model = trained_ncf_model

    if trained_model is None:
        print("Model NCF belum dilatih. Tidak dapat memberikan rekomendasi NCF.")
        return pd.DataFrame()

    # Siapkan input untuk semua lagu
    all_song_indices_encoded = df_ncf_rec['song_index_encoded'].values.reshape(-1, 1)
    artist_input_array = np.full((len(df_ncf_rec), 1), artist_id_encoded, dtype=int)
    additional_features_for_all_songs = df_ncf_rec[NCF_ADDITIONAL_FEATURES].values

    # Prediksi skor "popularitas" (atau kecocokan) untuk semua lagu bagi artis tersebut
    predicted_scores = trained_model.predict(
        [artist_input_array, all_song_indices_encoded, additional_features_for_all_songs],
        verbose=0
    ).flatten()

    # Tambahkan skor prediksi ke DataFrame sementara untuk diurutkan
    df_temp = df_ncf_rec.copy()
    df_temp['ncf_score'] = predicted_scores

    # Urutkan berdasarkan skor NCF (descending) dan ambil top_k
    top_recommendations = df_temp.sort_values(by='ncf_score', ascending=False).head(top_k)

    return top_recommendations[['id', 'name', 'artists', 'popularity', 'ncf_score'] + NCF_ADDITIONAL_FEATURES]


## 4.4. Sistem Rekomendasi Hybrid

def hybrid_recommend(df_hybrid, artist_id_encoded_hybrid, target_song_id_original_hybrid, top_k=10, ncf_weight=0.5, cbf_weight=0.5):
    global trained_ncf_model, df_global # Menggunakan model NCF global dan df_global

    if trained_ncf_model is None:
        print("Model NCF belum dilatih. Rekomendasi hybrid tidak dapat dijalankan.")
        return pd.DataFrame()

    if target_song_id_original_hybrid not in df_hybrid['id'].values:
        print(f"Lagu referensi dengan ID '{target_song_id_original_hybrid}' tidak ditemukan untuk hybrid.")
        return pd.DataFrame()

    # 1. Dapatkan Skor CBF
    # Skor CBF dihitung berdasarkan kemiripan dengan `target_song_id_original_hybrid`
    cbf_scores_series = content_based_similarity_scores(df_hybrid, target_song_id_original_hybrid, CBF_FEATURES)
    if cbf_scores_series.empty:
        print("Tidak bisa mendapatkan skor CBF untuk rekomendasi hybrid.")
        return pd.DataFrame()
    # Normalisasi Min-Max untuk skor CBF agar sebanding dengan output sigmoid NCF (0-1)
    if np.ptp(cbf_scores_series) > 1e-9:
         cbf_scores_normalized = (cbf_scores_series - cbf_scores_series.min()) / (cbf_scores_series.max() - cbf_scores_series.min())
    else:
         cbf_scores_normalized = pd.Series(np.zeros(len(df_hybrid)), index=df_hybrid.index)


    # 2. Dapatkan Skor NCF
    # Skor NCF dihitung untuk `artist_id_encoded_hybrid` terhadap semua lagu
    all_song_indices_encoded_hybrid = df_hybrid['song_index_encoded'].values.reshape(-1, 1)
    artist_input_array_hybrid = np.full((len(df_hybrid), 1), artist_id_encoded_hybrid, dtype=int)
    additional_features_all_songs_hybrid = df_hybrid[NCF_ADDITIONAL_FEATURES].values

    ncf_scores_array = trained_ncf_model.predict(
        [artist_input_array_hybrid, all_song_indices_encoded_hybrid, additional_features_all_songs_hybrid],
        verbose=0
    ).flatten()
    ncf_scores_series = pd.Series(ncf_scores_array, index=df_hybrid.index)

    # 3. Gabungkan Skor
    # Pastikan kedua series memiliki indeks yang sama (seharusnya sudah jika berasal dari df_hybrid)
    hybrid_scores = (ncf_weight * ncf_scores_series) + (cbf_weight * cbf_scores_normalized)

    # 4. Dapatkan Rekomendasi Top-K
    top_indices_hybrid = hybrid_scores.sort_values(ascending=False).index

    # Kecualikan lagu input (target_song_id_original_hybrid) dari rekomendasi
    input_song_idx_hybrid = df_hybrid[df_hybrid['id'] == target_song_id_original_hybrid].index
    if not input_song_idx_hybrid.empty:
        top_indices_hybrid = top_indices_hybrid.drop(input_song_idx_hybrid, errors='ignore')

    recommendations_df_hybrid = df_hybrid.iloc[top_indices_hybrid[:top_k]][['id', 'name', 'artists', 'popularity']]
    recommendations_df_hybrid['hybrid_score'] = hybrid_scores.iloc[top_indices_hybrid[:top_k]]
    recommendations_df_hybrid['cbf_score_norm'] = cbf_scores_normalized.iloc[top_indices_hybrid[:top_k]]
    recommendations_df_hybrid['ncf_score'] = ncf_scores_series.iloc[top_indices_hybrid[:top_k]]

    return recommendations_df_hybrid



Ringkasan Arsitektur Model NCF:


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input_artist   │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input_song_in… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 64)     │  2,143,232 │ user_input_artis… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_embedding      │ (None, 1, 64)     │ 10,575,040 │ item_input_song_… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_flatten        │ (None, 64)        │          0 │ user_embedding[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_flatten        │ (None, 64)        │          0 │ item_embedding[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ additional_feature… │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_layer   │ (None, 133)       │          0 │ user_flatten[0][… │
│ (Concatenate)       │                   │            │ item_flatten[0][… │
│                     │                   │            │ additional_featu… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fc1 (Dense)         │ (None, 128)       │     17,152 │ concatenate_laye… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout1 (Dropout)  │ (None, 128)       │          0 │ fc1[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fc2 (Dense)         │ (None, 64)        │      8,256 │ dropout1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout2 (Dropout)  │ (None, 64)        │          0 │ fc2[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, 1)         │         65 │ dropout2[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,743,745 (48.61 MB)

 Trainable params: 12,743,745 (48.61 MB)

 Non-trainable params: 0 (0.00 B)

## 4. Pemodelan

### 4.1. Model NCF (Neural Collaborative Filtering)

Model NCF bertujuan untuk mempelajari interaksi antara "pengguna" (dalam kasus ini, direpresentasikan oleh `artist_encoded`) dan "item" (direpresentasikan oleh `song_index_encoded`), dengan mempertimbangkan fitur-fitur tambahan dari lagu. Model ini dilatih untuk tugas klasifikasi biner, yaitu memprediksi apakah sebuah lagu akan populer (berdasarkan `is_popular_binary`).

**Arsitektur Model (`create_ncf_model`)**:

* **Input Layers**:
    * `user_input_artist`: Menerima ID artis yang sudah di-encode (`artist_encoded`). Bentuk: `(1,)`.
    * `item_input_song_index`: Menerima indeks lagu yang sudah di-encode (`song_index_encoded`). Bentuk: `(1,)`.
    * `additional_features_input`: Menerima vektor fitur numerik tambahan dari lagu (sesuai `NCF_ADDITIONAL_FEATURES`, contoh: `valence`, `danceability`, dll.). Jumlah fitur ditentukan oleh `num_additional_features` (default 5). Bentuk: `(num_additional_features,)`.

* **Embedding Layers**:
    * `user_embedding`: Mengubah ID artis menjadi vektor representasi padat (vektor laten). Ukuran input adalah `num_unique_artists`, dan dimensi output adalah `latent_dim` (default 64). Dilengkapi dengan regularisasi L2 (`embeddings_regularizer=l2(0.001)`) untuk mencegah overfitting.
    * `item_embedding`: Mengubah indeks lagu menjadi vektor laten. Ukuran input adalah `num_unique_songs`, dan dimensi output adalah `latent_dim`. Juga menggunakan regularisasi L2.

* **Flatten Layers**:
    * `user_flat` dan `item_flat`: Meratakan output dari embedding layers menjadi vektor satu dimensi agar dapat digabungkan dengan fitur lain.

* **Concatenation Layer**:
    * `concatenate_layer`: Menggabungkan output `user_flat`, `item_flat`, dan `additional_features_input` menjadi satu vektor fitur gabungan.

* **Dense Layers (Fully Connected Layers)**:
    * `fc1`: Layer Dense pertama dengan 128 neuron, fungsi aktivasi ReLU, dan regularisasi kernel L2 (`kernel_regularizer=l2(0.001)`).
    * `dropout1`: Layer Dropout dengan rate 0.4 setelah `fc1` untuk mengurangi overfitting.
    * `fc2`: Layer Dense kedua dengan 64 neuron, aktivasi ReLU, dan regularisasi kernel L2.
    * `dropout2`: Layer Dropout dengan rate 0.2 setelah `fc2`.

* **Output Layer**:
    * `output_layer`: Layer Dense tunggal dengan 1 neuron dan fungsi aktivasi Sigmoid. Menghasilkan probabilitas (nilai antara 0 dan 1) yang merepresentasikan kemungkinan lagu tersebut populer.

**Kompilasi Model**:
Model NCF dikompilasi dengan konfigurasi berikut:
* **Optimizer**: `Adam` dengan laju pembelajaran (learning rate) default 0.001.
* **Loss Function**: `binary_crossentropy`, sesuai untuk tugas klasifikasi biner.
* **Metrics**:
    * `accuracy`: Akurasi klasifikasi.
    * `KerasPrecision` (diberi nama alias `precision`): Presisi klasifikasi.
    * `KerasRecall` (diberi nama alias `recall`): Recall klasifikasi.

Inisialisasi model (`ncf_model_instance`) dilakukan dengan memanggil `create_ncf_model` menggunakan jumlah artis unik dan lagu unik dari `df_global`, serta jumlah fitur tambahan yang ditentukan.

### 4.2. Model Content-Based Filtering (CBF)

Pendekatan CBF merekomendasikan item berdasarkan kemiripan fitur intrinsik antara item itu sendiri. Jika pengguna menyukai suatu item, maka item lain dengan fitur serupa akan direkomendasikan.

**Perhitungan Skor Kemiripan (`content_based_similarity_scores`)**:
* **Input**: `df_cb` (DataFrame sumber data), `target_song_id_original` (ID string lagu target), `features_for_cbf` (daftar nama kolom fitur yang digunakan untuk perbandingan, contoh: `valence`, `danceability`, dll.).
* **Proses**:
    1.  Mengambil vektor fitur dari `target_song_id_original`.
    2.  Mengambil matriks fitur dari semua lagu dalam `df_cb`.
    3.  Menghitung jarak Euclidean antara vektor fitur lagu target dan matriks fitur semua lagu menggunakan `sklearn.metrics.pairwise.euclidean_distances`. Semakin kecil jarak, semakin mirip lagunya.
    4.  Mengkonversi jarak menjadi skor kemiripan menggunakan formula: $S = \frac{1}{1 + \text{jarak}}$. Skor yang lebih tinggi menandakan kemiripan yang lebih besar.
* **Output**: Sebuah Pandas Series berisi skor kemiripan untuk setiap lagu di `df_cb` relatif terhadap lagu target.

**Generasi Rekomendasi CBF Murni (`recommend_content_based_pure`)**:
* **Input**: `df_rec_cb` (DataFrame sumber), `target_song_id_original`, `top_k` (jumlah rekomendasi yang diinginkan).
* **Proses**:
    1.  Memanggil `content_based_similarity_scores` untuk mendapatkan skor kemiripan.
    2.  Mengurutkan lagu-lagu berdasarkan skor kemiripan secara menurun.
    3.  Mengecualikan `target_song_id_original` dari daftar jika ada.
    4.  Mengambil `top_k` lagu teratas.
* **Output**: DataFrame berisi `top_k` lagu yang direkomendasikan beserta detailnya (ID, nama, artis, popularitas, dan fitur CBF yang digunakan).

### 4.3. Fungsi Rekomendasi NCF Murni (Top-N)

Selain untuk klasifikasi, model NCF yang telah dilatih juga dapat digunakan untuk menghasilkan daftar rekomendasi Top-N untuk seorang artis.

**Generasi Rekomendasi NCF Murni (`recommend_ncf_pure`)**:
* **Input**: `df_ncf_rec` (DataFrame sumber), `artist_id_encoded` (ID artis yang telah di-encode), `top_k`, `trained_model` (model NCF yang telah dilatih).
* **Proses**:
    1.  Menyiapkan data input untuk prediksi NCF:
        * Array ID artis (`artist_input_array`): `artist_id_encoded` diulang sebanyak jumlah lagu di `df_ncf_rec`.
        * Array indeks semua lagu (`all_song_indices_encoded`): Semua `song_index_encoded` dari `df_ncf_rec`.
        * Array fitur tambahan semua lagu (`additional_features_for_all_songs`): Fitur-fitur `NCF_ADDITIONAL_FEATURES` dari `df_ncf_rec`.
    2.  Menggunakan `trained_model.predict()` untuk menghasilkan skor prediksi (probabilitas popularitas atau kecocokan) untuk setiap lagu di `df_ncf_rec` bagi `artist_id_encoded` yang diberikan.
    3.  Menambahkan skor prediksi ini ke DataFrame sementara.
    4.  Mengurutkan lagu berdasarkan `ncf_score` secara menurun.
    5.  Mengambil `top_k` lagu teratas.
* **Output**: DataFrame berisi `top_k` lagu yang direkomendasikan untuk artis tersebut, beserta detail dan skor NCF-nya.

### 4.4. Sistem Rekomendasi Hybrid

Sistem hybrid ini dirancang untuk menggabungkan kekuatan dari pendekatan NCF (yang menangkap pola kolaboratif dan preferensi laten) dan CBF (yang menangkap kemiripan berdasarkan atribut item).

**Generasi Rekomendasi Hybrid (`hybrid_recommend`)**:
* **Input**: `df_hybrid` (DataFrame sumber, bisa jadi subset kandidat saat evaluasi), `artist_id_encoded_hybrid` (ID artis untuk NCF), `target_song_id_original_hybrid` (ID lagu referensi untuk CBF), `top_k`, `ncf_weight` (bobot untuk skor NCF, default 0.5), `cbf_weight` (bobot untuk skor CBF, default 0.5).
* **Proses**:
    1.  **Dapatkan Skor CBF**:
        * Memanggil `content_based_similarity_scores` untuk menghitung kemiripan antara `target_song_id_original_hybrid` dan semua lagu dalam `df_hybrid`.
        * Menormalisasi skor CBF ini menggunakan Min-Max scaling agar berada dalam rentang [0, 1], sebanding dengan output sigmoid dari NCF. Ini dilakukan jika rentang nilai skor CBF (`np.ptp`) cukup besar untuk menghindari pembagian dengan nol.
    2.  **Dapatkan Skor NCF**:
        * Mempersiapkan input untuk model NCF (serupa dengan `recommend_ncf_pure`) untuk `artist_id_encoded_hybrid` terhadap semua lagu dalam `df_hybrid`.
        * Melakukan prediksi menggunakan `trained_ncf_model` untuk mendapatkan skor NCF (sudah dalam rentang [0, 1] karena aktivasi sigmoid).
    3.  **Gabungkan Skor**:
        * Menghitung skor hybrid sebagai kombinasi linear (rata-rata tertimbang) dari `ncf_scores_series` dan `cbf_scores_normalized`:
            `hybrid_scores = (ncf_weight * ncf_scores_series) + (cbf_weight * cbf_scores_normalized)`
    4.  **Dapatkan Rekomendasi Top-K**:
        * Mengurutkan lagu-lagu berdasarkan `hybrid_scores` secara menurun.
        * Mengecualikan `target_song_id_original_hybrid` dari daftar rekomendasi.
        * Mengambil `top_k` lagu teratas.
* **Output**: DataFrame berisi `top_k` lagu yang direkomendasikan secara hybrid, beserta detail lagu dan skor individual (hybrid, CBF ternormalisasi, NCF) untuk analisis lebih lanjut.

In [80]:
# Pelatihan dan Evaluasi Model NCF (untuk Tugas Klasifikasi)

def train_and_evaluate_ncf_classifier(df_full_for_training, train_df_for_eval, test_df_for_eval):
    global trained_ncf_model # Memodifikasi model global

    num_unique_artists_train = df_full_for_training['artist_encoded'].nunique()
    num_unique_songs_train = df_full_for_training['song_index_encoded'].nunique() # Menggunakan song_index_encoded

    # Buat instance model NCF baru untuk dilatih
    ncf_model_to_train = create_ncf_model(num_unique_artists_train, num_unique_songs_train, num_additional_features=len(NCF_ADDITIONAL_FEATURES))

    # Persiapan data training NCF
    X_train_artist = train_df_for_eval['artist_encoded'].values
    X_train_song_index = train_df_for_eval['song_index_encoded'].values # Menggunakan song_index_encoded
    X_train_features = train_df_for_eval[NCF_ADDITIONAL_FEATURES].values
    y_train_popularity = train_df_for_eval['is_popular_binary'].values # Target klasifikasi

    # Persiapan data validasi NCF (dari data test split sebelumnya)
    X_val_artist = test_df_for_eval['artist_encoded'].values
    X_val_song_index = test_df_for_eval['song_index_encoded'].values
    X_val_features = test_df_for_eval[NCF_ADDITIONAL_FEATURES].values
    y_val_popularity = test_df_for_eval['is_popular_binary'].values

    # Callbacks
    early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        mode='max',
        restore_best_weights=True
    )

    print("\nMemulai Pelatihan Model NCF...")
    history = ncf_model_to_train.fit(
        [X_train_artist, X_train_song_index, X_train_features],
        y_train_popularity,
        epochs=10,
        batch_size=64,
        validation_data=([X_val_artist, X_val_song_index, X_val_features], y_val_popularity),
        verbose=1,
        callbacks=[early_stopping]
    )

    print("\nRiwayat Metrik Training NCF (Klasifikasi):")
    if history and history.history:
        for epoch_num in range(len(history.history['loss'])):
            log_msg = f"Epoch {epoch_num + 1}: "
            log_msg += f"Loss = {history.history['loss'][epoch_num]:.4f}, "
            log_msg += f"Accuracy = {history.history['accuracy'][epoch_num]:.4f}, "
            log_msg += f"Precision = {history.history.get('precision', [0.0])[epoch_num]:.4f}, " # .get untuk kompatibilitas jika metrik tidak ada
            log_msg += f"Recall = {history.history.get('recall', [0.0])[epoch_num]:.4f}, "
            log_msg += f"Val Loss = {history.history['val_loss'][epoch_num]:.4f}, "
            log_msg += f"Val Accuracy = {history.history['val_accuracy'][epoch_num]:.4f}, "
            log_msg += f"Val Precision = {history.history.get('val_precision', [0.0])[epoch_num]:.4f}, "
            log_msg += f"Val Recall = {history.history.get('val_recall', [0.0])[epoch_num]:.4f}"
            print(log_msg)
    else:
        print("Tidak ada riwayat training yang tercatat untuk NCF.")

    trained_ncf_model = ncf_model_to_train # Simpan model yang sudah dilatih ke variabel global
    return history


# Latih model NCF untuk tugas klasifikasi
# df_global adalah DataFrame yang sudah diproses sepenuhnya
# train_df dan test_df adalah hasil split dari df_global
ncf_training_history = train_and_evaluate_ncf_classifier(df_global, train_df, test_df)

# 6. Evaluasi Kualitas Rekomendasi Top-N (Peringkat)

def get_ground_truth_for_song(df_source, target_song_id_original):
    """
    Mendapatkan ground truth untuk sebuah lagu: lagu lain dari artis yang sama di df_source.
    """
    target_song_info = df_source[df_source['id'] == target_song_id_original]
    if target_song_info.empty:
        return []
    target_artist_name = target_song_info['artists'].iloc[0]

    # Ambil semua lagu dari artis yang sama di df_source, kecuali lagu input itu sendiri
    ground_truth_songs = df_source[
        (df_source['artists'] == target_artist_name) & (df_source['id'] != target_song_id_original)
    ]['id'].tolist()
    return ground_truth_songs


def precision_recall_at_k(recommendations_ids, ground_truth_ids, k):
    """Menghitung Precision@K dan Recall@K."""
    recommended_k = recommendations_ids[:k]
    relevant_items = set(ground_truth_ids)
    recommended_relevant = [item for item in recommended_k if item in relevant_items]

    precision = len(recommended_relevant) / k if k > 0 else 0.0
    recall = len(recommended_relevant) / len(relevant_items) if len(relevant_items) > 0 else 0.0
    return precision, recall

def ndcg_at_k(recommendations_ids, ground_truth_ids, k):

    # Jika item ada di ground_truth_ids, relevansinya 1, jika tidak 0.
    relevance_scores = [[1 if item in ground_truth_ids else 0 for item in recommendations_ids[:k]]]
    # Jika ground_truth_ids kosong, semua item yang direkomendasikan tidak relevan.
    ideal_relevance = [[1] * min(k, len(ground_truth_ids))] # Skor relevansi ideal

    if not ground_truth_ids: # Jika tidak ada item relevan sama sekali
        return 0.0
    if not recommendations_ids: # Jika tidak ada rekomendasi
        return 0.0
    actual_relevance_for_ndcg = np.array(relevance_scores)

    if actual_relevance_for_ndcg.shape[1] == 0: # Jika tidak ada rekomendasi (k=0 atau list kosong)
        return 0.0

    return ndcg_score(actual_relevance_for_ndcg, np.arange(actual_relevance_for_ndcg.shape[1], 0, -1).reshape(1, -1), k=k)


def evaluate_ranking_metrics(df_evaluation_set, recommendation_function, recommender_name, top_k_eval, **kwargs):
    """Mengevaluasi metrik peringkat untuk fungsi rekomendasi tertentu."""
    all_precisions = []
    all_recalls = []
    all_ndcgs = []
    sample_eval_df = df_evaluation_set.sample(min(10, len(df_evaluation_set)), random_state=42)
    print(f"\nMengevaluasi {recommender_name} pada {len(sample_eval_df)} sampel lagu dari set evaluasi...")

    for i, row in sample_eval_df.iterrows():
        target_song_id_original_eval = row['id']
        target_artist_encoded_eval = row['artist_encoded'] # Diperlukan untuk NCF dan Hybrid

        # Dapatkan rekomendasi
        if recommender_name == "CBF Murni":
            recommendations_df = recommendation_function(df_global, target_song_id_original_eval, top_k=top_k_eval)
        elif recommender_name == "NCF Murni":
            # NCF murni butuh artist_id_encoded, bukan song_id
            recommendations_df = recommendation_function(df_global, target_artist_encoded_eval, top_k=top_k_eval)
        elif recommender_name == "Hybrid":
            recommendations_df = recommendation_function(df_global, target_artist_encoded_eval, target_song_id_original_eval, top_k=top_k_eval)
        else:
            continue

        if recommendations_df.empty:
            all_precisions.append(0.0)
            all_recalls.append(0.0)
            all_ndcgs.append(0.0)
            continue

        recommendations_ids_eval = recommendations_df['id'].tolist()

        ground_truth_ids_eval = get_ground_truth_for_song(df_global, target_song_id_original_eval)

        if not ground_truth_ids_eval and not recommendations_ids_eval:
             pass

        p_at_k, r_at_k = precision_recall_at_k(recommendations_ids_eval, ground_truth_ids_eval, top_k_eval)
        ndcg_val = ndcg_at_k(recommendations_ids_eval, ground_truth_ids_eval, top_k_eval)

        all_precisions.append(p_at_k)
        all_recalls.append(r_at_k)
        all_ndcgs.append(ndcg_val)

    avg_precision = np.mean(all_precisions) if all_precisions else 0.0
    avg_recall = np.mean(all_recalls) if all_recalls else 0.0
    avg_ndcg = np.mean(all_ndcgs) if all_ndcgs else 0.0

    print(f"\n--- Hasil Evaluasi Peringkat untuk {recommender_name} (K={top_k_eval}) ---")
    print(f"Rata-rata Precision@{top_k_eval}: {avg_precision:.4f}")
    print(f"Rata-rata Recall@{top_k_eval}: {avg_recall:.4f}")
    print(f"Rata-rata NDCG@{top_k_eval}: {avg_ndcg:.4f}")
    return avg_precision, avg_recall, avg_ndcg


if trained_ncf_model is None:
    print("Model NCF belum dilatih. Harap latih terlebih dahulu.")
else:
    # Evaluasi untuk CBF Murni
    evaluate_ranking_metrics(test_df, recommend_content_based_pure, "CBF Murni", K_EVALUATION)

    # Evaluasi untuk Hybrid
    evaluate_ranking_metrics(test_df, hybrid_recommend, "Hybrid", K_EVALUATION)
    print("\nEvaluasi Peringkat untuk NCF Murni diskip sementara karena definisi Ground Truth yang berbeda.")
    print("NCF telah dievaluasi pada metrik klasifikasi (akurasi, presisi, recall).")




Memulai Pelatihan Model NCF...
Epoch 1/10
2066/2066 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.7759 - loss: 0.9914 - precision: 0.7703 - recall: 0.7894 - val_accuracy: 0.8149 - val_loss: 0.4999 - val_precision: 0.8226 - val_recall: 0.8011
Epoch 2/10
2066/2066 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.8239 - loss: 0.4904 - precision: 0.8268 - recall: 0.8195 - val_accuracy: 0.8259 - val_loss: 0.4657 - val_precision: 0.8132 - val_recall: 0.8446
Epoch 3/10
2066/2066 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.8387 - loss: 0.4501 - precision: 0.8424 - recall: 0.8312 - val_accuracy: 0.8331 - val_loss: 0.4546 - val_precision: 0.8457 - val_recall: 0.8131
Epoch 4/10
2066/2066 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.8493 - loss: 0.4234 - precision: 0.8583 - recall: 0.8354 - val_accuracy: 0.8343 - val_loss: 0.4367 - val_precision: 0.8493 - val_recall: 0.8112
Epoch 5/10
2066/2066 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8547 - loss: 0.4013 - precision: 0.8651 - recal

### 5. Pelatihan dan Evaluasi Model NCF (untuk Tugas Klasifikasi)

Setelah model NCF didefinisikan, langkah berikutnya adalah melatihnya menggunakan data yang telah disiapkan dan mengevaluasi kinerjanya pada tugas klasifikasi utama, yaitu memprediksi apakah sebuah lagu akan populer.

**Fungsi `train_and_evaluate_ncf_classifier`**:

Fungsi ini mengelola seluruh proses pelatihan dan evaluasi performa klasifikasi model NCF.

* **Tujuan**: Melatih model NCF untuk memprediksi `is_popular_binary` dan mencatat metrik performa klasifikasi selama proses pelatihan.
* **Input**:
    * `df_full_for_training`: DataFrame keseluruhan yang telah diproses (biasanya `df_global`), digunakan untuk mendapatkan jumlah unik artis dan lagu.
    * `train_df_for_eval`: DataFrame latih (`train_df`).
    * `test_df_for_eval`: DataFrame uji (`test_df`), digunakan sebagai set validasi selama pelatihan NCF.
* **Proses Utama**:
    1.  **Inisialisasi Model**:
        * Menghitung jumlah unik artis (`num_unique_artists_train`) dan lagu (`num_unique_songs_train`) dari `df_full_for_training`. Informasi ini krusial untuk menentukan dimensi input pada layer Embedding di model NCF.
        * Membuat instance baru dari model NCF (`ncf_model_to_train`) dengan memanggil `create_ncf_model` menggunakan jumlah unik artis, lagu, dan jumlah fitur tambahan yang telah ditentukan (`NCF_ADDITIONAL_FEATURES`).
    2.  **Persiapan Data Training dan Validasi**:
        * **Data Training**:
            * `X_train_artist`: Nilai `artist_encoded` dari `train_df_for_eval`.
            * `X_train_song_index`: Nilai `song_index_encoded` dari `train_df_for_eval`.
            * `X_train_features`: Fitur-fitur tambahan (sesuai `NCF_ADDITIONAL_FEATURES`) dari `train_df_for_eval`.
            * `y_train_popularity`: Nilai target `is_popular_binary` dari `train_df_for_eval`.
        * **Data Validasi**: Disiapkan dengan cara serupa dari `test_df_for_eval` (`X_val_artist`, `X_val_song_index`, `X_val_features`, `y_val_popularity`). Data validasi ini digunakan untuk memantau kinerja model pada data yang tidak terlihat selama setiap epoch pelatihan, membantu dalam mendeteksi overfitting dan dalam pengambilan keputusan oleh callback `EarlyStopping`.
    3.  **Callbacks**:
        * `EarlyStopping`: Mekanisme untuk menghentikan pelatihan secara otomatis jika metrik yang dipantau (`monitor='val_accuracy'`) tidak menunjukkan peningkatan (`mode='max'`) selama jumlah epoch tertentu (`patience=10`). Opsi `restore_best_weights=True` memastikan bahwa bobot model dikembalikan ke kondisi terbaik yang dicapai pada metrik yang dipantau selama pelatihan.
        * `ModelCheckpoint` (disebutkan dalam komentar kode): Callback ini dapat diaktifkan untuk menyimpan bobot model terbaik secara periodik atau setiap kali ada peningkatan pada metrik yang dipantau.
    4.  **Pelatihan Model (`model.fit()`)**:
        * Model `ncf_model_to_train` dilatih menggunakan data training yang telah disiapkan.
        * `epochs=10`: Menentukan jumlah maksimum iterasi pelatihan. Dalam praktik, `EarlyStopping` dapat menghentikan pelatihan lebih awal.
        * `batch_size=64`: Jumlah sampel data yang diproses sebelum bobot model diperbarui.
        * `validation_data`: Set validasi yang digunakan untuk evaluasi di akhir setiap epoch.
        * `verbose=1`: Menampilkan log kemajuan pelatihan.
        * `callbacks=[early_stopping]`: Menggunakan callback `EarlyStopping` yang telah dikonfigurasi.
    5.  **Pencatatan Riwayat Metrik**:
        * Setelah pelatihan selesai, fungsi ini mencetak riwayat metrik untuk setiap epoch, termasuk `Loss`, `Accuracy`, `Precision`, dan `Recall` untuk data training, serta `Val Loss`, `Val Accuracy`, `Val Precision`, dan `Val Recall` untuk data validasi. Penggunaan `.get()` pada kamus `history.history` memastikan kode tetap berjalan meskipun salah satu metrik (misalnya, `precision` atau `recall`) tidak tercatat karena alasan tertentu.
    6.  **Pembaruan Model Global**: Model yang telah dilatih (`ncf_model_to_train`) disimpan ke dalam variabel global `trained_ncf_model` agar dapat diakses oleh fungsi rekomendasi lainnya.
* **Output**: Mengembalikan objek `history` yang berisi catatan semua metrik selama proses pelatihan.

Selanjutnya, skrip memanggil fungsi `train_and_evaluate_ncf_classifier` menggunakan `df_global` (sebagai `df_full_for_training`), `train_df` (sebagai `train_df_for_eval`), dan `test_df` (sebagai `test_df_for_eval`) untuk memulai proses pelatihan model NCF.

### 6. Evaluasi Kualitas Rekomendasi Top-N (Peringkat)

**Konsep Ground Truth**:
Untuk mengevaluasi kualitas peringkat, kita memerlukan "ground truth", yaitu daftar item yang dianggap benar-benar relevan untuk pengguna atau item input tertentu dalam set pengujian.

* **Untuk CBF & Hybrid (berbasis lagu input)**: Strategi yang digunakan adalah mendefinisikan lagu-lagu yang relevan sebagai **lagu lain dari artis yang sama** dengan lagu input yang diambil dari `test_df`. Ini adalah pendekatan sederhana untuk personalisasi berbasis artis.
* **Untuk NCF Murni (berbasis artis input)**: Dinyatakan bahwa evaluasi peringkat untuk NCF murni diskip sementara karena kompleksitas dalam mendefinisikan ground truth yang adil dan sebanding. Model NCF saat ini dilatih untuk memprediksi popularitas (`is_popular_binary`), sehingga ground truth peringkatnya bisa beragam (misalnya, lagu populer lain dari artis yang sama, atau lagu populer dari artis lain untuk tujuan *discovery*).

**Fungsi-fungsi Pendukung Evaluasi**:

1.  **`get_ground_truth_for_song(df_source, target_song_id_original)`**:
    * **Tujuan**: Mengidentifikasi dan mengembalikan daftar ID lagu yang dianggap sebagai ground truth untuk `target_song_id_original`.
    * **Proses**: Mencari nama artis dari `target_song_id_original` dalam `df_source`. Kemudian, mengumpulkan semua ID lagu lain dari artis yang sama yang ada di `df_source`, tidak termasuk `target_song_id_original` itu sendiri.
    * **Output**: List berisi ID lagu ground truth.

2.  **`precision_recall_at_k(recommendations_ids, ground_truth_ids, k)`**:
    * **Tujuan**: Menghitung metrik Precision@K dan Recall@K.
    * **Proses**:
        * Precision@K: Proporsi item yang direkomendasikan dalam Top-K yang relevan (ada di `ground_truth_ids`). Formula: $\frac{|\text{Top-K} \cap \text{GroundTruth}|}{K}$.
        * Recall@K: Proporsi item relevan yang berhasil ditemukan dalam Top-K rekomendasi. Formula: $\frac{|\text{Top-K} \cap \text{GroundTruth}|}{|\text{GroundTruth}|}$.
    * **Output**: Nilai `precision` dan `recall` pada `k`.

3.  **`ndcg_at_k(recommendations_ids, ground_truth_ids, k)`**:
    * **Tujuan**: Menghitung Normalized Discounted Cumulative Gain (NDCG@K), sebuah metrik yang menilai kualitas peringkat dengan memberi bobot lebih pada item relevan yang muncul di posisi lebih awal dalam daftar rekomendasi.
    * **Proses**: Menggunakan implementasi `sklearn.metrics.ndcg_score`. Daftar `recommendations_ids` diubah menjadi skor relevansi biner (1 jika relevan, 0 jika tidak) berdasarkan `ground_truth_ids`. Skor NDCG kemudian dihitung dengan membandingkan "cumulative gain" dari daftar yang direkomendasikan dengan "ideal discounted cumulative gain" (IDCG).
    * **Output**: Nilai `NDCG@K`.

**Fungsi Utama Evaluasi Peringkat (`evaluate_ranking_metrics`)**:

* **Tujuan**: Mengorkestrasi proses evaluasi peringkat untuk fungsi rekomendasi tertentu (CBF atau Hybrid).
* **Input**:
    * `df_evaluation_set`: DataFrame yang digunakan untuk mengambil sampel item uji (misalnya, `test_df`).
    * `recommendation_function`: Fungsi yang akan dievaluasi (misalnya, `recommend_content_based_pure` atau `hybrid_recommend`).
    * `recommender_name`: String nama sistem rekomendasi untuk keperluan logging.
    * `top_k_eval`: Nilai `K` untuk metrik @K.
* **Proses**:
    1.  Mengambil sampel acak (dalam kode terakhir, `min(10, len(df_evaluation_set))`) dari `df_evaluation_set` untuk mempercepat proses evaluasi.
    2.  Untuk setiap lagu (`row`) dalam sampel tersebut:
        * Mendapatkan `target_song_id_original_eval` dan `target_artist_encoded_eval`.
        * Memanggil `recommendation_function` yang diberikan (CBF atau Hybrid) untuk menghasilkan Top-K rekomendasi. `df_global` digunakan sebagai basis data lagu untuk fungsi rekomendasi.
        * Menangani kasus jika tidak ada rekomendasi yang dihasilkan.
        * Memanggil `get_ground_truth_for_song` untuk mendapatkan daftar lagu relevan dari `df_global`.
        * Menghitung `precision_at_k`, `recall_at_k`, dan `ndcg_at_k` menggunakan fungsi-fungsi pendukung.
        * Menyimpan skor metrik dari setiap item sampel.
    3.  Setelah semua item sampel diproses, menghitung rata-rata dari semua skor Precision@K, Recall@K, dan NDCG@K yang terkumpul.
    4.  Mencetak hasil rata-rata metrik evaluasi.
* **Output**: Tuple berisi rata-rata Precision@K, Recall@K, dan NDCG@K.

Terakhir, skrip memanggil `evaluate_ranking_metrics` untuk sistem "CBF Murni" dan "Hybrid", menggunakan `test_df` sebagai sumber item evaluasi dan `K_EVALUATION` sebagai nilai `K`. Seperti yang disebutkan sebelumnya, evaluasi peringkat untuk "NCF Murni" diskip dalam implementasi ini.

In [82]:
"""# 7. Inference dan Contoh Rekomendasi"""

def recommend_by_song_title_all_methods(df_all, song_title_input, top_k_inf=5):
    global le_artists_global, trained_ncf_model, df_global

    if trained_ncf_model is None or le_artists_global is None or df_global is None:
        print("Komponen sistem (model NCF, encoder artis, atau df global) belum siap.")
        return

    # Cari lagu berdasarkan judul
    potential_songs = df_all[df_all['name'].str.contains(song_title_input, case=False, na=False)]
    if potential_songs.empty:
        print(f"Lagu dengan judul yang mengandung '{song_title_input}' tidak ditemukan.")
        return

    # Ambil lagu pertama yang cocok sebagai referensi
    selected_song = potential_songs.iloc[0]
    song_id_original_ref = selected_song['id']
    song_name_ref = selected_song['name']
    song_artist_ref = selected_song['artists']

    print(f"\n--- Merekomendasikan berdasarkan lagu: '{song_name_ref}' oleh '{song_artist_ref}' (ID: {song_id_original_ref}) ---")

    # Dapatkan artist_encoded untuk NCF dan Hybrid
    try:
        # artist_encoded_ref = le_artists_global.transform([song_artist_ref])[0]
        artist_encoded_ref = selected_song['artist_encoded']

    except ValueError:
        print(f"Artis '{song_artist_ref}' tidak ditemukan dalam encoder. Mungkin artis ini hanya ada di baris yang di-drop.")
        return

    # 1. Rekomendasi NCF Murni
    print(f"\n--- Top-{top_k_inf} Rekomendasi NCF Murni (untuk artis: {song_artist_ref} [encoded: {artist_encoded_ref}]) ---")
    # df_global digunakan sebagai basis data untuk NCF merekomendasikan
    ncf_recommendations = recommend_ncf_pure(df_global, artist_encoded_ref, top_k=top_k_inf)
    if not ncf_recommendations.empty:
        # Kecualikan lagu input jika muncul dalam rekomendasi NCF
        ncf_recommendations = ncf_recommendations[ncf_recommendations['id'] != song_id_original_ref]
        print(ncf_recommendations[['name', 'artists', 'popularity', 'ncf_score']].head(top_k_inf))
    else:
        print("Tidak ada rekomendasi NCF yang dihasilkan.")

    # 2. Rekomendasi CBF Murni
    print(f"\n--- Top-{top_k_inf} Rekomendasi Content-Based Filtering (CBF) Murni ---")
    cbf_recommendations = recommend_content_based_pure(df_global, song_id_original_ref, top_k=top_k_inf)
    if not cbf_recommendations.empty:
        # CBF sudah mengecualikan lagu input secara internal
        print(cbf_recommendations[['name', 'artists', 'popularity'] + CBF_FEATURES[:2]]) # Tampilkan beberapa fitur CBF juga
    else:
        print("Tidak ada rekomendasi CBF yang dihasilkan.")

    # 3. Rekomendasi Hybrid
    print(f"\n--- Top-{top_k_inf} Rekomendasi Hybrid ---")
    hybrid_recommendations = hybrid_recommend(df_global, artist_encoded_ref, song_id_original_ref, top_k=top_k_inf)
    if not hybrid_recommendations.empty:
        # Hybrid sudah mengecualikan lagu input secara internal
        print(hybrid_recommendations[['name', 'artists', 'popularity', 'hybrid_score', 'cbf_score_norm', 'ncf_score']])
    else:
        print("Tidak ada rekomendasi Hybrid yang dihasilkan.")


# Contoh Penggunaan Inference
if trained_ncf_model and le_artists_global and df_global is not None:
    print("\n=============== CONTOH INFERENCE REKOMENDASI ===============")
    if not df_global.empty:
        # Pilih lagu sampel secara acak atau berdasarkan indeks tertentu
        # sample_song_title = df_global['name'].iloc[12345] # Pastikan indeks valid
        sample_song_title = df_global.sample(1, random_state=42)['name'].iloc[0] # Ambil satu judul lagu acak
        print(f"Mencari rekomendasi berdasarkan kemiripan dengan lagu yang judulnya mengandung: '{sample_song_title}'")
        recommend_by_song_title_all_methods(df_global, sample_song_title, top_k_inf=5)


    else:
        print("DataFrame global kosong, tidak dapat memilih lagu sampel untuk rekomendasi.")
else:
    print("\nModel NCF atau komponen lain belum siap. Tidak dapat menjalankan contoh inference.")



=============== CONTOH INFERENCE REKOMENDASI ===============
Mencari rekomendasi berdasarkan kemiripan dengan lagu yang judulnya mengandung: 'Kite'

--- Merekomendasikan berdasarkan lagu: 'Being For The Benefit Of Mr. Kite! - Remastered 2009' oleh '['The Beatles']' (ID: 6W35n1UlkvqhfMZstB4BXs) ---

--- Top-5 Rekomendasi NCF Murni (untuk artis: ['The Beatles'] [encoded: 28670]) ---
                                           name  \
88843                               Joan of Arc   
147207                         Whole Lotta Weed   
147774                                 Hood Hop   
51521   Whistle While You Twurk (ColliPark Mix)   
71812                           Kings Never Die   

                                artists  popularity  ncf_score  
88843   ['Night Lovell', '$uicideBoy$']        0.65   0.988991  
147207                  ['Project Pat']        0.36   0.988954  
147774                       ['J-Kwon']        0.37   0.988665  
51521               ['Ying Yang Twins']        0

### 7. Inference dan Contoh Rekomendasi

Bagian ini mendemonstrasikan bagaimana sistem rekomendasi yang telah dibangun dan dilatih dapat digunakan untuk menghasilkan rekomendasi lagu secara praktis (proses inference). Fungsi utamanya adalah `recommend_by_song_title_all_methods`, yang memungkinkan pengguna mendapatkan rekomendasi berdasarkan judul lagu yang diinput.

**Fungsi `recommend_by_song_title_all_methods`**:

Fungsi ini dirancang untuk memberikan pengalaman pengguna yang mudah dengan mencari lagu berdasarkan judul dan kemudian menghasilkan rekomendasi dari ketiga metode yang telah diimplementasikan: NCF Murni, CBF Murni, dan Hybrid.

* **Tujuan**: Menyediakan antarmuka tunggal untuk mendapatkan rekomendasi lagu dari berbagai model berdasarkan input judul lagu.
* **Input**:
    * `df_all`: DataFrame yang berisi keseluruhan data lagu yang telah diproses (umumnya `df_global`).
    * `song_title_input`: String yang berisi judul lagu (atau bagian dari judul lagu) yang ingin dijadikan referensi.
    * `top_k_inf`: Jumlah rekomendasi teratas yang ingin ditampilkan untuk setiap metode (default adalah 5).
* **Proses Utama**:
    1.  **Pengecekan Komponen Sistem**: Memastikan bahwa komponen global penting seperti `trained_ncf_model` (model NCF yang sudah dilatih), `le_artists_global` (encoder untuk artis), dan `df_global` (DataFrame utama) sudah siap dan tidak `None`. Jika salah satu belum siap, fungsi akan mencetak pesan kesalahan dan berhenti.
    2.  **Pencarian Lagu Referensi**:
        * Mencari lagu dalam `df_all` yang kolom `name`-nya mengandung `song_title_input`. Pencarian ini bersifat *case-insensitive* (tidak membedakan huruf besar/kecil) dan mengabaikan nilai `NaN`.
        * Jika tidak ada lagu yang cocok ditemukan, fungsi akan mencetak pesan bahwa lagu tidak ditemukan dan berhenti.
        * Jika ditemukan satu atau lebih lagu, lagu pertama yang cocok (`potential_songs.iloc[0]`) akan dipilih sebagai `selected_song` (lagu referensi).
        * Informasi penting dari `selected_song` diekstrak: `id` (sebagai `song_id_original_ref`), `name` (sebagai `song_name_ref`), dan `artists` (sebagai `song_artist_ref`).
    3.  **Pengambilan `artist_encoded`**:
        * `artist_encoded_ref` (ID artis yang telah di-encode) untuk lagu referensi diambil langsung dari kolom `artist_encoded` pada `selected_song`. Ini adalah pendekatan yang lebih robust karena kolom `artists` bisa berisi daftar artis sebagai string (misalnya, "['Artis A', 'Artis B']") yang mungkin tidak cocok dengan cara `le_artists_global` dilatih (biasanya pada nama artis individual).
    4.  **Generasi Rekomendasi per Metode**:
        * **Rekomendasi NCF Murni**:
            * Mencetak header yang mengindikasikan rekomendasi NCF untuk artis dari lagu referensi.
            * Memanggil fungsi `recommend_ncf_pure(df_global, artist_encoded_ref, top_k=top_k_inf)` untuk mendapatkan rekomendasi.
            * Jika rekomendasi berhasil dibuat:
                * Lagu referensi (`song_id_original_ref`) akan dikecualikan dari daftar rekomendasi NCF jika kebetulan muncul.
                * Mencetak `top_k_inf` rekomendasi teratas beserta detail (`name`, `artists`, `popularity`, `ncf_score`).
            * Jika tidak ada rekomendasi NCF, pesan yang sesuai akan dicetak.
        * **Rekomendasi CBF Murni (Content-Based Filtering)**:
            * Mencetak header untuk bagian CBF.
            * Memanggil `recommend_content_based_pure(df_global, song_id_original_ref, top_k=top_k_inf)`.
            * Jika rekomendasi berhasil dibuat:
                * (Fungsi CBF sudah dirancang untuk mengecualikan lagu input secara internal).
                * Mencetak `top_k_inf` rekomendasi teratas beserta detail (`name`, `artists`, `popularity`, dan dua fitur pertama yang digunakan untuk CBF sebagai contoh).
            * Jika tidak ada rekomendasi CBF, pesan yang sesuai akan dicetak.
        * **Rekomendasi Hybrid**:
            * Mencetak header untuk bagian Hybrid.
            * Memanggil `hybrid_recommend(df_global, artist_encoded_ref, song_id_original_ref, top_k=top_k_inf)`.
            * Jika rekomendasi berhasil dibuat:
                * (Fungsi Hybrid juga sudah dirancang untuk mengecualikan lagu input secara internal).
                * Mencetak `top_k_inf` rekomendasi teratas beserta detail (`name`, `artists`, `popularity`, `hybrid_score`, `cbf_score_norm`, `ncf_score`) untuk memberikan wawasan tentang bagaimana skor gabungan terbentuk.
            * Jika tidak ada rekomendasi Hybrid, pesan yang sesuai akan dicetak.

**Contoh Penggunaan Inference (Blok Kode Eksekusi)**:

Blok kode di akhir bagian ini menunjukkan cara menggunakan fungsi `recommend_by_song_title_all_methods`.

1.  **Pengecekan Kesiapan**: Pertama, dilakukan pengecekan apakah `trained_ncf_model`, `le_artists_global`, dan `df_global` sudah tersedia.
2.  **Pemilihan Lagu Sampel**:
    * Jika semua komponen siap dan `df_global` tidak kosong, sebuah lagu sampel dipilih secara acak dari `df_global` menggunakan `df_global.sample(1, random_state=42)['name'].iloc[0]`. Penggunaan `random_state=42` memastikan lagu sampel yang dipilih akan sama setiap kali kode dijalankan (reproducible).
    * Terdapat juga contoh yang dikomentari untuk memilih lagu berdasarkan indeks baris tertentu (`df_global['name'].iloc[12345]`).
3.  **Pemanggilan Fungsi Rekomendasi Utama**:
    * Fungsi `recommend_by_song_title_all_methods` dipanggil dengan `df_global`, judul lagu sampel yang telah dipilih, dan `top_k_inf=5` untuk mendapatkan 5 rekomendasi teratas dari setiap metode.
4.  **Contoh Tambahan (Dikomentari)**: Ada placeholder yang dikomentari untuk mencoba rekomendasi dengan judul lagu spesifik, misalnya `"Love Story"`.
5.  **Penanganan Kasus Khusus**: Terdapat blok `else` untuk menangani situasi jika `df_global` kosong atau jika komponen sistem belum siap, dengan mencetak pesan informatif.